In [ ]:
train=pd.read_csv('/Users/namchangseok/cp1/train.csv')
test=pd.read_csv('/Users/namchangseok/cp1/test.csv')
sub=pd.read_csv('/Users/namchangseok/cp1/sample_submission.csv')


In [ ]:
# 데이터 전처리 

# 이상치 제거 
child_num_cut = 4
train.loc[train['child_num'] > child_num_cut, 'child_num'] = child_num_cut # child num의 수가 4 이상이면 이상치라 판단하여 제거 
famil_size_cut=6
train.loc[train['family_size']> famil_size_cut,'family_size']=famil_size_cut# family size 가 6 이상이면 이상치라 판단하여 제거 

# 의미 없는값 제거 
#train=train.drop(['FLAG_MOBIL'],axis=1)   # 개수가 전체 행과 같음 즉 의미가 없는 특성임

# 전처리 
for data in [train,test]:
    data['gender']=data['gender'].replace(['F','M'], [0, 1])
    data['car']=data['car'].replace(['N','Y'], [0, 1])
    data['reality']=data['reality'].replace(['N','Y'], [0, 1])
    data['edu_type'] = data['edu_type'].replace(['Higher education', 'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree'], [4, 2, 3, 1 ,5])
    
    
def DAYS_BIRTH(df):
    df['DAYS_BIRTH'] = df['DAYS_BIRTH'].apply(abs) / 365
    df['DAYS_BIRTH'] = df['DAYS_BIRTH'].apply(round)
    df['DAYS_BIRTH'] = np.select([df['DAYS_BIRTH'] < 10, df['DAYS_BIRTH'] < 20, df['DAYS_BIRTH'] < 30, df['DAYS_BIRTH'] < 40, df['DAYS_BIRTH'] < 50, df['DAYS_BIRTH'] < 60, df['DAYS_BIRTH'] < 70, df['DAYS_BIRTH'] >= 70],
                                [0, 10, 20, 30, 40, 50, 60, 70])
    return df['DAYS_BIRTH'] #

# 데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미. 양수 값은 고용되지 않은 상태를 의미함
def DAYS_EMPLOYED(df): 
    condition = (df['DAYS_EMPLOYED'] > 0)
    df.loc[condition, 'DAYS_EMPLOYED'] = 0
    df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].apply(abs) / 365
    df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].apply(round)
    return df['DAYS_EMPLOYED'] # 일 시작한지 '몇 년' 됐는지 리턴. 1년 단위로 계산

def begin_month(df):
    df['begin_month'] = df['begin_month'].apply(abs) / 12
    df['begin_month'] = df['begin_month'].apply(round)
    return df['begin_month'] # 신용카드 발급한 지 '몇 년' 됐는지 리턴



train['DAYS_BIRTH'] = DAYS_BIRTH(train)
train['DAYS_EMPLOYED'] = DAYS_EMPLOYED(train)
train['begin_month'] = begin_month(train)
train= train.rename(columns={'DAYS_BIRTH':'AGE', 'DAYS_EMPLOYED':"WORKING_YEARS",'begin_month':'YEAR_USE_CREDIT'})

test['DAYS_BIRTH'] = DAYS_BIRTH(test)
test['DAYS_EMPLOYED'] = DAYS_EMPLOYED(test)
test['begin_month'] = begin_month(test)
test= test.rename(columns={'DAYS_BIRTH':'AGE', 'DAYS_EMPLOYED':"WORKING_YEARS",'begin_month':'YEAR_USE_CREDIT'})


In [ ]:
# 의미 없는값 제거 
train=train.drop(['FLAG_MOBIL'],axis=1)   # 개수가 전체 행과 같음 즉 의미가 없는 특성임
test=test.drop(['FLAG_MOBIL'],axis=1)
# 모델 임시로 돌리기 위해 직업값 제거
train = train.drop("occyp_type", axis = 1)
test =  test.drop("occyp_type", axis = 1)

In [ ]:
# 새로운 특성 합쳐만들어보기 

train['youngrich']=[None for i in range(len(train['reality']))]
train[(train['reality']==1)&(train['AGE']<30)&(train['car']==1)]['youngrich']=1
train[(train['reality']!=1)|(train['AGE']>=30)|(train['car']!=1)]['youngrich']=0